In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-text-classification1/data/c1_train.txt"
devDataPath = "s3://corpus-text-classification1/data/c1_dev.txt"
testDataPath = "s3://corpus-text-classification1/data/c1_test.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.50d.txt"
savePath = "s3://corpus-text-classification1/c1_biLSTM/c1_biLSTM"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    contents, labels = [], []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            try:
                word = line.lower().strip().split()
                label = int(word[0].split(":")[0])
                content = word[1:]
                
                contents.append(content)
                labels.append(label)
            except:
                pass
    return contents, labels


def readCategory():
    """读取分类目录，固定id"""
    '''
    Retrieve Value
    Filter
    Compute Derived Value
    Find Extremum
    Sort
    Determine Range
    Characterize Distribution
    Find Anomalies
    Cluster
    Correlate
    '''
    categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
    cat_to_id = dict(zip(categories, range(1,len(categories)+1)))
    id_to_cat = dict(zip(range(1,len(categories)+1), categories))
    return id_to_cat, cat_to_id


def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd

In [4]:
contents_train, labels_train = readfile(trainDataPath)
contents_dev, labels_dev = readfile(devDataPath)
contents_test, labels_test = readfile(testDataPath)

In [5]:
id_to_cat, cat_to_id = readCategory()
num_classes = len(id_to_cat)

In [6]:
num_classes, labels_train[0]

(10, 1)

In [7]:
contents_all = contents_train + contents_dev + contents_test
seq_length = 0
for content in contents_all:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 35

In [8]:
seq_length

35

In [9]:
vocab, embd = loadGloVe(vocabPath, 50)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loading GloVe!
Completed!


In [10]:
word_to_id = dict(zip(vocab, range(vocab_size)))

In [11]:
len(embedding),embedding_dim,vocab_size

(400000, 50, 400000)

In [12]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [13]:
print("Loading training and validation and testing data...")
start_time = time.time()
x_train, y_train = process_file(contents_train, labels_train, word_to_id, num_classes, seq_length)  # seq_length = 600
x_dev, y_dev = process_file(contents_dev, labels_dev, word_to_id, num_classes, seq_length)
x_test, y_test = process_file(contents_test, labels_test, word_to_id, num_classes, seq_length)
time_dif = get_time_dif(start_time)
print("Loading data Time usage:", time_dif)

Loading training and validation and testing data...
Loading data Time usage: 0:00:00


In [14]:
x_train[0], y_train[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  102,   14,    0,  485,    3, 4791,   13,  375, 2693,
         812,  188], dtype=int32),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [15]:
contents_train[0]

['what',
 'is',
 'the',
 'population',
 'of',
 'arkansas',
 'on',
 'july',
 '1st',
 '2002',
 '?']

In [16]:
word_to_id["arkansas"]

4791

In [17]:
embedding[3]

array([ 0.70853  ,  0.57088  , -0.4716   ,  0.18048  ,  0.54449  ,
        0.72603  ,  0.18157  , -0.52393  ,  0.10381  , -0.17566  ,
        0.078852 , -0.36216  , -0.11829  , -0.83336  ,  0.11917  ,
       -0.16605  ,  0.061555 , -0.012719 , -0.56623  ,  0.013616 ,
        0.22851  , -0.14396  , -0.067549 , -0.38157  , -0.23698  ,
       -1.7037   , -0.86692  , -0.26704  , -0.2589   ,  0.1767   ,
        3.8676   , -0.1613   , -0.13273  , -0.68881  ,  0.18444  ,
        0.0052464, -0.33874  , -0.078956 ,  0.24185  ,  0.36576  ,
       -0.34727  ,  0.28483  ,  0.075693 , -0.062178 , -0.38988  ,
        0.22902  , -0.21617  , -0.22562  , -0.093918 , -0.80375  ])

In [18]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        '''
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        '''
        self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [19]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 30
batch_size = 128
print_per_batch = 20  # 每多少轮输出一次结果

sess = tf.Session()
lstm = AdversarailLSTM(embedding)
saver = tf.train.Saver()

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [20]:
globalStep = tf.Variable(0, name="globalStep", trainable=False)
# 定义优化函数，传入学习速率参数
optimizer = tf.train.AdamOptimizer(learning_rate)
# 计算梯度,得到梯度和变量
gradsAndVars = optimizer.compute_gradients(lstm.loss)
# 将梯度应用到变量下，生成训练器
trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

# 准确率
correct_pred = tf.equal(tf.argmax(lstm.inputY, 1), lstm.y_pred_cls)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初始化所有变量
sess.run(tf.global_variables_initializer())

In [21]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, x_pad, y_pad, loss1, acc1):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {lstm.inputX: x_batch1, lstm.inputY: y_batch1, lstm.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len

In [22]:
print('Training and evaluating...')
start_time = time.time()
total_batch = 0  # 总批次
best_acc_val = 0.0  # 最佳验证集准确率
last_improved = 0  # 记录上一次提升批次
require_improvement = 300  # 如果超过500轮未提升，提前结束训练
flag = False

for epoch in range(num_epochs):
    print('Epoch:', epoch + 1)
    batch_train = batch_iter(x_train, y_train, batch_size)
    for x_batch, y_batch in batch_train:
        feed_dict = {lstm.inputX: x_batch, lstm.inputY: y_batch, lstm.dropoutKeepProb: dropout_keep_prob}
        sess.run(trainOp, feed_dict=feed_dict)  # 运行优化
        total_batch += 1

        if total_batch % print_per_batch == 0:
            # 每多少轮次输出在训练集和验证集上的性能
            feed_dict[lstm.dropoutKeepProb] = 1.0
            loss_train, acc_train = sess.run([lstm.loss, acc], feed_dict=feed_dict)
            loss_val, acc_val = evaluate(sess, x_dev, y_dev, lstm.loss, acc)
            if acc_val > best_acc_val:
                # 保存最好结果
                best_acc_val = acc_val
                last_improved = total_batch
                saver.save(sess=sess, save_path=savePath)
                improved_str = '*'  # 对最好结果进行标记
            else:
                improved_str = ''

            time_dif = get_time_dif(start_time)
            msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%},' \
                  + ' Val Loss: {3:>6.2}, Val Acc: {4:>7.2%}, Time: {5} {6}'
            print(msg.format(total_batch, loss_train, acc_train, loss_val, acc_val, time_dif, improved_str))

        if total_batch - last_improved > require_improvement:
            # 验证集正确率长期不提升，提前结束训练
            print("No optimization for a long time, auto-stopping...")
            flag = True
            break  # 跳出循环
    if flag:  # 同上
        break

Training and evaluating...
Epoch: 1
Iter:     20, Train Loss:    2.2, Train Acc:  18.75%, Val Loss:    2.3, Val Acc:  16.78%, Time: 0:00:33 *
Iter:     40, Train Loss:    2.2, Train Acc:  17.19%, Val Loss:    2.1, Val Acc:  22.38%, Time: 0:01:00 *
Epoch: 2
Iter:     60, Train Loss:    2.0, Train Acc:  27.34%, Val Loss:    2.0, Val Acc:  33.15%, Time: 0:01:26 *
Iter:     80, Train Loss:    1.8, Train Acc:  35.94%, Val Loss:    1.8, Val Acc:  38.88%, Time: 0:01:53 *
Epoch: 3
Iter:    100, Train Loss:    1.6, Train Acc:  49.22%, Val Loss:    1.7, Val Acc:  41.26%, Time: 0:02:20 *
Iter:    120, Train Loss:    1.5, Train Acc:  51.56%, Val Loss:    1.5, Val Acc:  49.79%, Time: 0:02:47 *
Epoch: 4
Iter:    140, Train Loss:    1.4, Train Acc:  56.25%, Val Loss:    1.5, Val Acc:  52.73%, Time: 0:03:13 *
Iter:    160, Train Loss:    1.4, Train Acc:  53.12%, Val Loss:    1.4, Val Acc:  53.01%, Time: 0:03:36 *
Iter:    180, Train Loss:    1.4, Train Acc:  50.51%, Val Loss:    1.3, Val Acc:  59.02%,

In [25]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
def evaluate_model():
    # 读取保存的模型
    saver.restore(sess=sess, save_path=savePath)
    start_time = time.time()
    print('Testing...')
    loss_test, acc_test = evaluate(sess, x_test, y_test, lstm.loss, acc)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    test_data_len = len(x_test)
    test_num_batch = int((test_data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)  # 获得类别
    y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

    for i in range(test_num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, test_data_len)
        feed_dict = {
            lstm.inputX: x_test[start_id:end_id],
            lstm.dropoutKeepProb: 1.0
        }
        y_test_pred_cls[start_id:end_id] = sess.run(lstm.y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
    '''
    sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
        y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
        y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
        labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
        target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
        原文链接：https://blog.csdn.net/akadiao/article/details/78788864
    '''

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
    '''
    混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
    这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
    https://blog.csdn.net/u011734144/article/details/80277225
    '''
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

In [26]:
evaluate_model()

INFO:tensorflow:Restoring parameters from s3://corpus-text-classification1/c1_biLSTM/c1_biLSTM


INFO:tensorflow:Restoring parameters from s3://corpus-text-classification1/c1_biLSTM/c1_biLSTM


Testing...
Test Loss:   0.67, Test Acc:  81.31%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.90      0.77      0.83        71
                   Filter       0.83      0.71      0.77        77
    Compute Derived Value       0.69      0.78      0.73        79
            Find Extremum       0.90      0.88      0.89        84
                     Sort       0.80      0.83      0.81        63
          Determine Range       0.73      0.81      0.77        64
Characterize Distribution       0.83      0.85      0.84        68
           Find Anomalies       0.84      0.78      0.81        69
                  Cluster       0.78      0.86      0.82        65
                Correlate       0.86      0.84      0.85        77

                micro avg       0.81      0.81      0.81       717
                macro avg       0.82      0.81      0.81       717
             weighted avg       0.82      0.8

In [ ]:
def predict(predict_sentences, word_to_id, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    
    data_id = []
    # 将文本内容转换为对应的id形式
    for i in range(len(predict_sentences)):
        data_id.append([word_to_id[x] for x in predict_sentences[i].lower().strip().split() if x in word_to_id])
        
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    feed_dict = {
        input_x: x_pad,
        keep_prob: 1.0
    }
    predict_result = session.run(y_pred_cls, feed_dict=feed_dict)
    predict_result = [i+1 for i in predict_result]
    return predict_result

In [ ]:
predict_sentences = ["In the sixtieth ceremony , where were all of the winners from ?",  #  7
                    "On how many devices has the app \" CF SHPOP ! \" been installed ?",  # 1
                    "List center - backs by what their transfer _ fee was ."]  # 5
predict(predict_sentences, word_to_id, seq_length)